This GUI crops an image by dragging a box around part of an image and creates a assets folder with those images

In [2]:
import tkinter as tk
from tkinter import filedialog, simpledialog
from PIL import Image, ImageTk
import os
import random
import string

In [4]:
class ImageLabelingApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Image Labeling Tool")
        self.root.geometry("1200x900")

        # Initialize self.images as an empty list
        self.images = []

        # Canvas Dimensions
        self.canvas_width = 1200
        self.canvas_height = 900

        

        # Frame for buttons
        button_frame = tk.Frame(root, bg='gray')
        button_frame.pack(side="bottom", fill="x")  # Then pack the frame for buttons

        # Styling buttons
        button_style = {'bg': 'lightblue', 'fg': 'black'}

        # Pack buttons and entry inside the button frame
        self.goto_image_entry = tk.Entry(button_frame, width=10)
        self.goto_image_entry.configure(state='normal')
        self.goto_image_entry.pack(side="left")

        self.total_images_label = tk.Label(button_frame, text="Total Images: Loading...", bg='gray')
        self.total_images_label.pack(side="left")

        self.goto_button = tk.Button(button_frame, text="Go To", command=self.goto_image, **button_style)
        self.goto_button.pack(side="left")

        self.delete_button = tk.Button(button_frame, text="Delete", command=self.delete_current_image, **button_style)
        self.delete_button.pack(side="left")

        self.reset_button = tk.Button(button_frame, text="Reset Box", command=self.reset_current_box, **button_style)
        self.reset_button.pack(side="left")

        self.previous_button = tk.Button(button_frame, text="Previous", command=self.previous_image, **button_style)
        self.previous_button.pack(side="left")

        self.skip_button = tk.Button(button_frame, text="Skip", command=self.skip_current_image, **button_style)
        self.skip_button.pack(side="left")

        self.confirm_button = tk.Button(button_frame, text="Confirm", command=self.save_cropped_image, **button_style)
        self.confirm_button.pack(side="left")

        # Canvas setup
        self.canvas = tk.Canvas(root, bg="black", width=self.canvas_width, height=self.canvas_height, cursor="cross")
        self.canvas.pack()  # Pack the canvas first

        # Variables initialization
        self.image_dir = filedialog.askdirectory(title="Select Image Directory")
        self.current_image_index = 0
        self.rect = None
        self.rect_label = None
        self.label = None
        self.start_x = None
        self.start_y = None

        # Load images and update the total_images_label
        self.load_images()
        self.total_images_label.config(text=f"Total Images: {len(self.images)}")
        self.show_image()

        # Bindings
        self.canvas.bind("<ButtonPress-1>", self.on_button_press)
        self.canvas.bind("<ButtonRelease-1>", self.on_button_release)

    def previous_image(self):
        if self.current_image_index > 1:
            self.current_image_index -= 2  # Adjusting for the increment in show_image
            self.show_image()
        else:
            print("No previous images.")

    # Define the goto_image function
    def goto_image(self):
        try:
            image_number = int(self.goto_image_entry.get()) - 1
            # print("Entered value:", image_number)  # Corrected variable name
            if 0 <= image_number < len(self.images):
                self.current_image_index = image_number
                self.show_image()
            else:
                print("Invalid image number. Please enter a number between 1 and", len(self.images))
        except ValueError:
            print("Please enter a valid number.")
            
    # Define the skip_current_image function
    def skip_current_image(self):
        if self.current_image_index < len(self.images):
            self.show_image()  # Display the next image
        else:
            print("No more images to skip.")

    def delete_current_image(self):
        if self.current_image_index > 0:
            current_image_path = self.images[self.current_image_index - 1]
            try:
                os.remove(current_image_path)  # Delete the current image file
                print(f"Deleted: {current_image_path}")
            except OSError as e:
                print(f"Error: {e.strerror}")
            
            # Remove the deleted image from the list and adjust the index
            del self.images[self.current_image_index - 1]
            self.current_image_index -= 1
        
        self.show_image()  # Display the next image
    
    def load_images(self):
        self.images = [os.path.join(self.image_dir, f) for f in os.listdir(self.image_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]


    def show_image(self):
        if self.current_image_index < len(self.images):
            # Update the total_images_label with current image index
            self.total_images_label.config(text=f"Image {self.current_image_index + 1} of {len(self.images)}")
            self.canvas.delete("all")  # Clear the previous image and box

            image_path = self.images[self.current_image_index]
            original_image = Image.open(image_path)

            # Calculate the new size maintaining aspect ratio
            ratio = min(self.canvas_width / original_image.width, self.canvas_height / original_image.height)
            new_width = int(original_image.width * ratio)
            new_height = int(original_image.height * ratio)

            # Resize the image for display
            display_image = original_image.resize((new_width, new_height), Image.ANTIALIAS)
            self.current_image = ImageTk.PhotoImage(display_image)

            # Calculate position to center the image
            x = (self.canvas_width - new_width) // 2
            y = (self.canvas_height - new_height) // 2
            self.canvas.create_image(x, y, anchor='nw', image=self.current_image)
            self.current_image_index += 1
        else:
            print("No more images.")
            self.root.quit()

    def on_button_press(self, event):
        # Start drawing the bounding box
        self.start_x = event.x
        self.start_y = event.y
        self.rect = self.canvas.create_rectangle(self.start_x, self.start_y, event.x, event.y, outline='red')
        # Bind the motion event to dynamically update the bounding box
        self.canvas.bind("<Motion>", self.on_mouse_move)

    def on_mouse_move(self, event):
        # Update the size of the rectangle as the mouse is moving
        self.canvas.coords(self.rect, self.start_x, self.start_y, event.x, event.y)

    def on_button_release(self, event):
        # Unbind the motion event
        self.canvas.unbind("<Motion>")

        # Finalize the size of the rectangle
        self.canvas.coords(self.rect, self.start_x, self.start_y, event.x, event.y)

        # Ask for label
        label = simpledialog.askstring("Label", "Enter the label:")
        if label:
            color = self.get_unique_color()
            self.canvas.itemconfig(self.rect, outline=color)

            # Create and store the label text element
            label_x = event.x + 10
            label_y = event.y - 20
            self.rect_label = self.canvas.create_text(label_x, label_y, text=label, anchor='nw', fill=color)
            self.label = label
        else:
            # Delete the rectangle if labeling is canceled
            self.reset_current_box()

    def get_unique_color(self):
        # Function to generate a unique color for each box
        # This can be as simple or complex as needed
        r = lambda: random.randint(0,255)
        return f'#{r():02x}{r():02x}{r():02x}'

    def reset_current_box(self):
        # Check if there is a current rectangle
        if self.rect:
            # Delete the rectangle from the canvas
            self.canvas.delete(self.rect)
            self.rect = None

            # Also delete the associated label, if any
            if self.rect_label:
                self.canvas.delete(self.rect_label)
                self.rect_label = None

    def save_cropped_image(self):
        label_dir = "assets"
        os.makedirs(label_dir, exist_ok=True)
        label = self.label
        if label and self.rect:
            # Get the coordinates of the rectangle on the canvas
            x1, y1, x2, y2 = self.canvas.coords(self.rect)

            # Open the original image (not resized)
            original_image = Image.open(self.images[self.current_image_index - 1])

            # Calculate the scaling factor
            ratio = min(self.canvas_width / original_image.width, self.canvas_height / original_image.height)

            # Calculate the scaled image size
            scaled_width = original_image.width * ratio
            scaled_height = original_image.height * ratio

            # Calculate the position offset if the image is centered
            offset_x = (self.canvas_width - scaled_width) / 2
            offset_y = (self.canvas_height - scaled_height) / 2

            # Adjust the coordinates considering the scaling and offset
            orig_x1 = max((x1 - offset_x) / ratio, 0)
            orig_y1 = max((y1 - offset_y) / ratio, 0)
            orig_x2 = min((x2 - offset_x) / ratio, original_image.width)
            orig_y2 = min((y2 - offset_y) / ratio, original_image.height)

            # Crop the original image using adjusted coordinates
            cropped_image = original_image.crop((orig_x1, orig_y1, orig_x2, orig_y2))

            # Generate a unique suffix for the filename
            unique_suffix = ''.join(random.choices(string.ascii_lowercase + string.digits, k=6))
            file_name = f'{label}_{unique_suffix}.jpg'

            cropped_image.save(f'{label_dir}/{file_name}')

            self.show_image()  # Load next image

In [5]:
# Create and start the GUI application
if __name__ == "__main__":
    root = tk.Tk()
    app = ImageLabelingApp(root)
    root.mainloop()

C:\Users\Grego\AppData\Local\Temp\ipykernel_9164\2787425223.py:132: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  display_image = original_image.resize((new_width, new_height), Image.ANTIALIAS)
